#install

In [2]:
%%time 

# Important library for many geopython libraries
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes 
# Install Folium for Geographic data visualization
!pip install folium
# Install plotlyExpress
!pip install plotly_express
!pip install Shapely

Reading package lists... Done
Building dependency tree       
Reading state information... Done
gdal-bin is already the newest version (2.2.3+dfsg-2).
python-gdal is already the newest version (2.2.3+dfsg-2).
The following additional packages will be installed:
  python3-numpy
Suggested packages:
  python-numpy-doc python3-nose python3-numpy-dbg
The following NEW packages will be installed:
  python3-gdal python3-numpy
0 upgraded, 2 newly installed, 0 to remove and 40 not upgraded.
Need to get 2,288 kB of archives.
After this operation, 13.2 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 python3-numpy amd64 1:1.13.3-2ubuntu1 [1,943 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python3-gdal amd64 2.2.3+dfsg-2 [346 kB]
Fetched 2,288 kB in 1s (2,409 kB/s)
Selecting previously unselected package python3-numpy.
(Reading database ... 148492 files and directories currently installed.)
Preparing to unpack .../python3-numpy_

In [3]:
from shapely.geometry import Point,Polygon
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt 
import folium
from folium import plugins
from folium.plugins import HeatMap
import plotly_express as px
from google.colab import drive
from IPython.display import display
import glob
import re
drive.mount('/content/drive')

Mounted at /content/drive


#read

In [4]:
district_NE_path = "/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/District_NE.geojson"

In [5]:
district_NE = gpd.read_file(district_NE_path)

#Algorithm

In [7]:
def check_district(lon,lat):
  point = Point(lon,lat)
  for index in district_NE.index:
    return point.within(district_NE.loc[index,'geometry'])

#Filter NE

In [11]:
!ls "/content/drive/My Drive/gps_copy/2019-04"

2019-04-01_00.zip  2019-04-08_12.zip  2019-04-16_00.zip  2019-04-23_12.zip
2019-04-01_03.zip  2019-04-08_15.zip  2019-04-16_03.zip  2019-04-23_15.zip
2019-04-01_06.zip  2019-04-08_18.zip  2019-04-16_06.zip  2019-04-23_18.zip
2019-04-01_09.zip  2019-04-08_21.zip  2019-04-16_09.zip  2019-04-23_21.zip
2019-04-01_12.zip  2019-04-09_00.zip  2019-04-16_12.zip  2019-04-24_00.zip
2019-04-01_15.zip  2019-04-09_03.zip  2019-04-16_15.zip  2019-04-24_03.zip
2019-04-01_18.zip  2019-04-09_06.zip  2019-04-16_18.zip  2019-04-24_06.zip
2019-04-01_21.zip  2019-04-09_09.zip  2019-04-16_21.zip  2019-04-24_09.zip
2019-04-02_00.zip  2019-04-09_12.zip  2019-04-17_00.zip  2019-04-24_12.zip
2019-04-02_03.zip  2019-04-09_15.zip  2019-04-17_03.zip  2019-04-24_15.zip
2019-04-02_06.zip  2019-04-09_18.zip  2019-04-17_06.zip  2019-04-24_18.zip
2019-04-02_09.zip  2019-04-09_21.zip  2019-04-17_09.zip  2019-04-24_21.zip
2019-04-02_12.zip  2019-04-10_00.zip  2019-04-17_12.zip  2019-04-25_00.zip
2019-04-02_15.zip  2019-0

In [27]:
all_files = glob.glob('/content/drive/My Drive/gps_copy/2019-04/' + '/*.zip')

In [28]:
x = 8
y = 16

In [29]:
all_files[x:y]

['/content/drive/My Drive/gps_copy/2019-04/2019-04-02_00.zip',
 '/content/drive/My Drive/gps_copy/2019-04/2019-04-02_03.zip',
 '/content/drive/My Drive/gps_copy/2019-04/2019-04-02_06.zip',
 '/content/drive/My Drive/gps_copy/2019-04/2019-04-02_09.zip',
 '/content/drive/My Drive/gps_copy/2019-04/2019-04-02_12.zip',
 '/content/drive/My Drive/gps_copy/2019-04/2019-04-02_15.zip',
 '/content/drive/My Drive/gps_copy/2019-04/2019-04-02_18.zip',
 '/content/drive/My Drive/gps_copy/2019-04/2019-04-02_21.zip']

In [31]:
for i in all_files[0:8]:
  print(i)

/content/drive/My Drive/gps_copy/2019-04/2019-04-01_00.zip
/content/drive/My Drive/gps_copy/2019-04/2019-04-01_03.zip
/content/drive/My Drive/gps_copy/2019-04/2019-04-01_06.zip
/content/drive/My Drive/gps_copy/2019-04/2019-04-01_09.zip
/content/drive/My Drive/gps_copy/2019-04/2019-04-01_12.zip
/content/drive/My Drive/gps_copy/2019-04/2019-04-01_15.zip
/content/drive/My Drive/gps_copy/2019-04/2019-04-01_18.zip
/content/drive/My Drive/gps_copy/2019-04/2019-04-01_21.zip


In [32]:
%time
all_files = glob.glob('/content/drive/My Drive/gps_copy/2019-04/' + '/*.zip')

# day 1
for filename in all_files[0:8]:
  gps = pd.read_csv(filename, compression='zip')

  # clean
  gps = gps[gps['unit_type'] != 1.0]
  gps = gps.dropna()

  # filter
  msk_NE = []
  for index,row in gps.iterrows():
    if check_district(row.lon,row.lat):
      msk_NE.append(True)
    else:
      msk_NE.append(False)

  gps = gps[msk_NE]

  # export
  filename = re.sub('zip$', 'csv', filename)
  gps.to_csv(filename, encoding='utf-8', index = False)

  print('complete')

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 9.78 µs
complete
complete
complete
complete
complete
complete
complete
complete
